<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase5/blob/main/Fase5_Tech_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 5 Tech Challenge

In [15]:
!git clone https://github.com/guilherme-argentino/fiap-ia4devs-techchallenge-fase5.git conteudo

Cloning into 'conteudo'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 146 (delta 2), reused 0 (delta 0), pack-reused 137 (from 2)
Receiving objects: 100% (146/146), 93.64 MiB | 32.11 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [16]:
!apt-get install git-lfs
!git lfs install

%cd /content/conteudo

!git lfs pull

%cd /content

!pip install -r conteudo/requirements.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Git LFS initialized.
/content/conteudo
/content
Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing ins

In [42]:
import os
import sys

import cv2
from ultralytics import YOLO

# Set YOLO to quiet mode
os.environ['YOLO_VERBOSE'] = 'False'

def detect_and_highlight_objects_in_photo(image_path):
    image_orig = cv2.imread(image_path)

    yolo_model = YOLO('yolov8n.pt')  # load an official model
    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')

    results = yolo_model(image_orig)

    for result in results:
        classes = result.names
        cls = result.boxes.cls
        conf = result.boxes.conf
        detections = result.boxes.xyxy

        for pos, detection in enumerate(detections):
            if conf[pos] >= 0.5:
                xmin, ymin, xmax, ymax = detection
                label = f"{classes[int(cls[pos])]} {conf[pos]:.2f}"
                color = (0, int(cls[pos]), 255)
                cv2.rectangle(image_orig, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                cv2.putText(image_orig, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    result_path = "./imgs/Test/teste.jpg"
    cv2.imwrite(result_path, image_orig)
    return result_path

def detect_objects_in_video(video_path):

    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')
    yolo_model.verbose = False
    video_capture = cv2.VideoCapture(video_path)

    # Flag to track if any objects have been detected
    object_detected = False

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        results = yolo_model(frame, verbose=False)

        for result in results:
          classes = result.names
          cls = result.boxes.cls
          conf = result.boxes.conf
          detections = result.boxes.xyxy
          if len(detections) > 0: # Check if any detections are made
            for pos, detection in enumerate(detections):
              print(f"{classes[int(cls[pos])]} {conf[pos]:.2f}")
            object_detected = True
            break  # Exit the loop if an object is detected

        if object_detected:
          break  # Exit the outer loop if an object is detected

    video_capture.release()

    return object_detected # Return True if object_detected is True, otherwise False

def detect_and_highlight_objects_in_video(video_path):
    yolo_model = YOLO('yolov8n.pt')  # load an official model
    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')
    video_capture = cv2.VideoCapture(video_path)
    width = int(video_capture.get(3))
    height = int(video_capture.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    result_video_path = "detected_objects_video1.avi"
    out = cv2.VideoWriter(result_video_path, fourcc, 20.0, (width, height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        results = yolo_model(frame)

        for result in results:
            classes = result.names
            cls = result.boxes.cls
            conf = result.boxes.conf
            detections = result.boxes.xyxy

            for pos, detection in enumerate(detections):
                if conf[pos] >= 0.5:
                    xmin, ymin, xmax, ymax = detection
                    label = f"{classes[int(cls[pos])]} {conf[pos]:.2f}"
                    color = (0, int(cls[pos]), 255)
                    cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                    cv2.putText(frame, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

        out.write(frame)
    video_capture.release()
    out.release()

    return result_video_path

def detect_objects_and_plot(path_orig):
    image_orig = cv2.imread(path_orig)

    yolo_model = YOLO('yolov8n.pt')  # load an official model
    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')

    results = yolo_model(image_orig)

    for result in results:
        classes = result.names
        cls = result.boxes.cls
        conf = result.boxes.conf
        detections = result.boxes.xyxy

        for pos, detection in enumerate(detections):
            if conf[pos] >= 0.5:
                xmin, ymin, xmax, ymax = detection
                label = f"{classes[int(cls[pos])]} {conf[pos]:.2f}"
                color = (0, int(cls[pos]), 255)
                cv2.rectangle(image_orig, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                cv2.putText(image_orig, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    cv2.imshow("Teste", image_orig)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [43]:
video_path = 'conteudo/video1.mp4'

result = detect_objects_in_video(video_path)

print("\n\n#####################################################################")
if result:
    print("\n\nObjeto cortante encontrado? Sim")
else:
    print("\n\nObjeto cortante encontrado? Não")
print("\n\n#####################################################################")


guns 0.30


#####################################################################


Objeto cortante encontrado? Sim


#####################################################################
